In [1]:
from torchvision.datasets.video_utils import VideoClips

import pandas as pd
import os
import time
import pickle
import seaborn as sns

from sklearn.model_selection import train_test_split
import copy

In [6]:
main_df = pd.read_csv('finalized_df.csv')

In [7]:
main_video_path = 'D:/Dataset_BENG/short_clips/all_short_clips/'
main_df['video_path'] = main_video_path + main_df['main_dir'].astype(str) + '/' + main_df['pig_id'].astype(str) + '/' + main_df['video'].astype(str)+'.avi'

In [8]:
robust_path = 'D:/Dataset_BENG/short_clips/all_short_clips/robust_test'

robust_clips = []
for root, dirs, files in os.walk('D:/Dataset_BENG/short_clips/all_short_clips/robust_test'):
    for file in files:
        clip = os.path.join(root, file)
        robust_clips.append(clip)

In [10]:
final_df = copy.deepcopy(main_df[~main_df['video_path'].isin(robust_clips)])
final_df_ = final_df[['video_path', 'label', 'category']].copy().reset_index(drop=True)
robust_df = copy.deepcopy(main_df[main_df['video_path'].isin(robust_clips)]) #Test Set 2 (manually selected)
robust_df = robust_df.reset_index(drop=True)

In [11]:
X, y = final_df_[['video_path', 'category']], final_df_['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, shuffle=True, random_state=42)

In [ ]:
# train set
train_df = X_train.merge(y_train.to_frame(), left_index=True, right_index=True)
train_df = train_df.reset_index(drop=True)

# Test Set 1
test_df = X_test.merge(y_test.to_frame(), left_index=True, right_index=True)
test_df = test_df.reset_index(drop=True)

print("train size:", len(train_df))
print("test size:", len(test_df))

In [ ]:
train_df.to_csv('train_data_random_final.csv', index=False)
test_df.to_csv('test_data_random_final.csv', index=False)

In [ ]:
# Test Set 2
robust_df_ = robust_df[['video_path', 'category', 'label']].copy()
robust_df_.to_csv('robust_test.csv', index=False)

### Convert into pickle file for train & test sets

In [ ]:
# video clip properties
clip_length_in_frames=60
frames_between_clips=60

#### Train Set

In [ ]:
start=time.time()
video_clips=VideoClips(video_paths=train_df['video_path'],
                      clip_length_in_frames=clip_length_in_frames,
                      frames_between_clips=frames_between_clips)
end=time.time()
total=end-start
print(f'Spent time: {total//60 :.0f}m {total%60 :.0f}s')

In [ ]:
with open('D:/Dataset_BENG/short_clips/all_short_clips/train_set.pkl', 'wb') as f:
    pickle.dump(video_clips, f)

#### Test Set 1

In [ ]:
start=time.time()
video_clips_=VideoClips(video_paths=test_df['video_path'],
                      clip_length_in_frames=clip_length_in_frames,
                      frames_between_clips=frames_between_clips)
end=time.time()
total=end-start
print(f'Spent time: {total//60 :.0f}m {total%60 :.0f}s')

In [ ]:
with open('D:/Dataset_BENG/short_clips/all_short_clips/test_set.pkl', 'wb') as f:
    pickle.dump(video_clips_, f)

#### Test Set 2

In [ ]:
start=time.time()
video_clips_r=VideoClips(video_paths=robust_df_['video_path'],
                      clip_length_in_frames=clip_length_in_frames,
                      frames_between_clips=frames_between_clips)
end=time.time()
total=end-start
print(f'Spent time: {total//60 :.0f}m {total%60 :.0f}s')

In [ ]:
with open('D:/Dataset_BENG/short_clips/all_short_clips/robust_test.pkl', 'wb') as f:
    pickle.dump(video_clips_r, f)